In [3]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

import json
import os

with open("../config.json") as fptr:
    config = json.load(fptr)
    os.environ["OPENAI_API_KEY"]  = config["OPENAI_API_KEY"]
    os.environ["OPENAI_API_BASE"] = config["OPENAI_API_BASE"]

llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

res = llm.invoke(messages)
print(res.content)

String theory is a theoretical framework in physics that aims to describe the fundamental particles and forces of nature. It suggests that the most basic entities in the universe are not point-like particles, but tiny, one-dimensional "strings" that vibrate in different ways.

In string theory, these tiny strings can vibrate at different frequencies, resulting in different particles and their corresponding properties. For example, a string vibrating at a certain frequency may appear as an electron, while a different vibration frequency may manifest as a photon or a quark.

One of the key features of string theory is that it attempts to unify the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces. This unification is achieved by incorporating additional dimensions beyond the familiar three spatial dimensions and one time dimension.

String theory also suggests that there may be multiple possible configurations or "compactifications"

In [4]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator

class Joke(BaseModel):
    setup: str = Field(..., description="question to setup a joke")
    punchline: str = Field(..., description="answer to resolve the joke")

    @field_validator('setup')
    @classmethod
    def question_ends_with_question_mark(cls, field: str):
        if not field.endswith('?'):
            raise ValueError(f"Badly formed question: {field}")
        return field
    
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

joke_query = "Tell me a joke."
model_input  = prompt.format_prompt(query=joke_query)
model_output = llm.invoke(model_input.to_string())

parser.parse(model_output.content)

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')